### 지도학습(Supervised Learning)
- 목표변수(target, Y) 있는 학습법 

In [53]:
import pandas as pd 

In [54]:
df_BCD = pd.read_csv('../../../datasets/BreastCancerWisconsinDataSet.csv') 
df_BCD[:2]

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.8,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.6,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.9,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.8,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN


#### 목표변수 - 연속형
- 목표변수 : radius_mean
- 설명변수 : texture_mean, perimeter_mean

In [55]:
df_BCD.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 33 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   id                       569 non-null    int64  
 1   diagnosis                569 non-null    object 
 2   radius_mean              569 non-null    float64
 3   texture_mean             569 non-null    float64
 4   perimeter_mean           569 non-null    float64
 5   area_mean                569 non-null    float64
 6   smoothness_mean          569 non-null    float64
 7   compactness_mean         569 non-null    float64
 8   concavity_mean           569 non-null    float64
 9   concave points_mean      569 non-null    float64
 10  symmetry_mean            569 non-null    float64
 11  fractal_dimension_mean   569 non-null    float64
 12  radius_se                569 non-null    float64
 13  texture_se               569 non-null    float64
 14  perimeter_se             5

In [56]:
df_BCD.columns

Index(['id', 'diagnosis', 'radius_mean', 'texture_mean', 'perimeter_mean',
       'area_mean', 'smoothness_mean', 'compactness_mean', 'concavity_mean',
       'concave points_mean', 'symmetry_mean', 'fractal_dimension_mean',
       'radius_se', 'texture_se', 'perimeter_se', 'area_se', 'smoothness_se',
       'compactness_se', 'concavity_se', 'concave points_se', 'symmetry_se',
       'fractal_dimension_se', 'radius_worst', 'texture_worst',
       'perimeter_worst', 'area_worst', 'smoothness_worst',
       'compactness_worst', 'concavity_worst', 'concave points_worst',
       'symmetry_worst', 'fractal_dimension_worst', 'Unnamed: 32'],
      dtype='object')

#### Preprocessing 전처리

##### 목표변수와 설명변수 추출 

In [57]:
df_BCD_extract = df_BCD[['radius_mean', 'texture_mean', 'perimeter_mean']]
df_BCD_extract[:2]

,radius_mean,texture_mean,perimeter_mean
0,17.99,10.38,122.8
1,20.57,17.77,132.9


#### 결측치나 이상치 확인 

In [58]:
df_BCD_extract.isnull().sum()

radius_mean       0
texture_mean      0
perimeter_mean    0
dtype: int64

##### Structured data 
- 목표변수, Y, **Target**
- 설명변수, X, **Label** 

In [59]:
target = df_BCD_extract['radius_mean'] #하나는 series -> list와 유사해서 열 표시가 되지 않는다 
labels = df_BCD_extract[['texture_mean', 'perimeter_mean']] #두개 이상은 dataframe
target.shape, labels.shape
#행, 열 

((569,), (569, 2))

#### 모델(알고리즘) 학습 


##### 모델 학습
- target datatype 종속 

In [60]:
from sklearn.linear_model import LinearRegression

In [61]:
model = LinearRegression()
model.fit(labels, target)

LinearRegression()

#### 평가 

In [62]:
target_predict = model.predict(labels) 
# 시험, 답 없이 학습된 항목만 
# 레코드의 갯수만큼 리스트로 출력 
len(target_predict)

569

In [63]:
from sklearn.metrics import r2_score
# r2_score 통해 평가를 통한 점수를 만들어낼 수 있다 
r2_score(target, target_predict)

0.9957437087622522

#### 미래예측(서비스 개시) 

In [64]:
df_BCD_extract[50:55]

,radius_mean,texture_mean,perimeter_mean
50,11.76,21.60,74.72
51,13.64,16.34,87.21
52,11.94,18.24,75.71
53,18.22,18.70,120.30
54,15.10,22.02,97.26


In [65]:
# 16.34, 87.21 : 외부에서 입력한 설명변수 값들 
temp_label = [[16.34, 87.21]] 
# 두가지 값이니까 dataframe처럼 두번 싸
# 머신런닝은 컬럼 이름을 받지 않고 숫자 부분만 받는다 
# 설명변수2개 넣고 목적변수를 예측 
# 결측치를 채우는 방식 (결측치를 목적변수로 해서) 

model.predict(temp_label)

#### 서비스 배포

In [67]:
import pickle 
#메모리 인스턴스 클래스를 이진파일로 저장 
#우리가 교육시켰던 모델(linear regression)은 인스턴스화 되었고 메모리에 올라가 있다(class)
#영구적으로 사용하려면 메모리에 올라가 있는 걸 파일로 저장해야하는데 그것을 피클로 함. 

In [70]:
with open('../../../datasets/BreastCancerWisconsin_Regression.pkl', 'wb') as regression_file:
    pickle.dump(obj=model, file=regression_file)
    pass 
#파이썬에선 클래스가 오브젝트. 우리는 모델을 오브젝트, 모델을 파일 wb
#이 파일을 인스턴스화해서 사용하면 교육한 거 사용 가능 
#경로 쓸때 유의점: 주피터랩은 프로젝트에 종속적이지 않고 독립적임./vscode 파이썬은 루트 기준

#### 일반화 확인 

In [74]:
model.intercept_, model.coef_ #texture_mean, perimeter_mean이 순서대로 나온 것임
# y= sigma(coef_*x) + intercept_ 
# y= (-0.00463664 *texture_mean) + (0.12298807*permimeter mean) + 0.8823180437943332

(0.8823180437943314, array([-0.00463664,  0.14498807]))

In [78]:
# [[16.34, 87.21]] 대입하면 
y=(-0.00463664 * 16.34) + (0.14498807 * 87.21) + 0.8823180437943332
y
# array([13.45096511]) 
# predict했을때와 같은 값이 나온다


13.450964930894331